In [1]:
import random

from src.data_generator.csv_data_processor import CSVDataProcessor
from src.utils.statistcs.statistical_functions import create_distributed_age_df
from spark_instance import spark
from src.utils.statistcs.data_visualisations import plot_age_distribution_with_sd, plot_kernel_density_age_distribution
from pyspark.sql.types import StructType, StructField, StringType, FloatType

from pyspark.sql.functions import col
  

24/05/07 22:15:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [ ]:
average_age = 40.2

csv_age_file_file = CSVDataProcessor(spark, "data/uk/uk_age_population.csv")

csv_age_uk_df = csv_age_file_file.runner()

In [2]:
import os
docker_indicators = ["/.dockerenv", "/proc/self/cgroup"]
docker_found = any(os.path.exists(indicator) for indicator in docker_indicators)
if docker_found:
    print("Running inside a Docker container.")
else:
    print("Running outside Docker.")


Running outside Docker.


In [2]:
spark.sql("USE healthcare")

In [2]:
from data_generator.base_data_creator import BaseDataCreator

bdc = BaseDataCreator(spark, "uk")
base_df = bdc.runner()

In [3]:
base_df.show()

+---+----------+----------+------+----------------+--------------------+--------+---------+------------+------------+--------------------+
|Age|       DOB|Blood_type|Gender|            Name|           Ethnicity|Postcode|is_female|is_pediatric|is_geriatric|           unique_id|
+---+----------+----------+------+----------------+--------------------+--------+---------+------------+------------+--------------------+
|  0|2024-04-25|        O-|  Male|      J. Edwards|         White other| EX6 3YJ|    false|        true|       false|J_Edwards|2024-04...|
|  0|2023-07-04|        A+|Female|    F. O. Taylor|       White English| OX4 3SY|     true|        true|       false|F_O_Taylor|2023-0...|
|  0|2023-09-25|        O-|  Male|     Z. McDonald|       White English| NE7 3XP|    false|        true|       false|Z_McDonald|2023-0...|
|  0|2023-12-02|        B+|  Male|        Y. Lyons|Mixed White/Black...| BL7 0WN|    false|        true|       false|Y_Lyons|2023-12-0...|
|  0|2023-08-09|        B+|

In [ ]:
# plot_age_distribution_with_sd(csv_age_uk_df, average_age)

In [ ]:
# plot_kernel_density_age_distribution(csv_age_uk_df)

In [ ]:
opt_df = base_df.repartition(8)

In [ ]:
import time

In [ ]:
start = time.time()
opt_df.write.format("parquet").mode("overwrite").saveAsTable("base_table_10000")
end = time.time()
print("time taken to save df: " + str(end - start))

In [ ]:
base_table_df = spark.sql("select * from base_table_10000")

In [ ]:
base_table_df.show()

In [ ]:
from src.constants.admission_types_dataset import admission_mapping
flattened = [
    (
        top_level.name,
        sub_level_key.name,
        list(sub_level_info.get("stay_types")),
        sub_level_info.get("tests"),  
        random.choice(sub_level_info.get("doctors"))
    )
    for top_level, sub_level_dict in admission_mapping.items()
    for sub_level_key, sub_level_info in sub_level_dict.items()
]



In [ ]:
flattened


In [ ]:
from src.constants.condition_probabilities import condition_age_probability_dict

flattened_condition_probabilities = [
    (
        sub_admission,
        condition,
        gender if gender in ['male', 'female'] else None,
        float(age_range[0]),
        float(age_range[1]) if len(age_range) > 1 else float('inf'),
        float(probability) 
    )
    for sub_admission, conditions in condition_age_probability_dict.items()
    for condition, genders_or_age_prob_list in conditions.items()
    for gender, age_prob_list in (genders_or_age_prob_list.items() if isinstance(genders_or_age_prob_list, dict) else [(None, genders_or_age_prob_list)])
    for age_range, probability in age_prob_list
]



In [ ]:

flattened_condition_probabilities

In [ ]:
columns = ["condition_admission_type", "condition", "gender", "age_min", "age_max", "probability"]
schema = StructType([
    StructField("condition_admission_type", StringType(), True),
    StructField("condition", StringType(), True),
    StructField("condition_gender", StringType(), True),  # Assuming gender can be specific probability for gender 'male', 'female', or null
    StructField("age_min", FloatType(), False),
    StructField("age_max", FloatType(), False),
    StructField("probability", FloatType(), False)
])

condition_prob_df = spark.createDataFrame(data=flattened_condition_probabilities, schema=schema)


In [ ]:
condition_prob_df.show()

In [ ]:
"""
STEP 1 
    From the flattened data list we wish to create a DataFrame. This contains all the possible combinations for the given 
    top level admissions, sub level admissions, stay types and list of tests available from the admission_mapping, stay_type and admission_tests lists or dictionaries in admission_types_test_dataset.py
"""

mapping_df = spark.createDataFrame(flattened, ["top_level_admission", "sub_level_admission", "stay_types", "possible_tests", "doctor"])

joined_tbl = mapping_df.join(condition_prob_df, on=[mapping_df.sub_level_admission == condition_prob_df.condition_admission_type], how="left")

In [ ]:
from functools import reduce
from typing import Type, Any
from concurrent.futures import ThreadPoolExecutor, as_completed
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import spark_partition_id


class TaskWrapper:
    def __init__(self, spark, task_class, *args, **kwargs):
        """

        Args:
            spark (SparkSession): The SparkSession
            task_class: The class that's going to be run.
            *args: The args associated with class.
            **kwargs: The kwargs associated with class, if any.
        """
        self.task = task_class(spark, *args, **kwargs)

    def runner(self) -> Any:
        """
        Calls the task runner function
        Returns:
            Any: returns Any type, right now DataFrame
        """
        return self.task.runner()


def process_partition(spark: SparkSession,
                      partition_id: int,
                      class_to_run: Type,
                      df: DataFrame,
                      *args,
                      **kwargs) -> DataFrame:
    """
    This processes the partition result of the spark_partition process result from the TaskWrapper.runner()
    Args:
        spark (SparkSession): The SparkSession
        partition_id (int): The partition id to filter against the df
        class_to_run (Type): The class to run
        df (DataFrame): The partitioned df 
        *args: The args associated with class.
        **kwargs: The kwargs associated with class, if any.
    Returns:
        Dataframe: The Dataframe result. 
    """
    # Filter the DataFrame for this partition's data
    partition_df = df.filter(spark_partition_id() == partition_id)

    # Update class instantiation with *args and **kwargs
    task = TaskWrapper(spark, class_to_run, partition_df, *args[1:], **kwargs)

    result_df = task.runner()
    return result_df


def run_multithreaded_processing(spark: SparkSession,
                                 class_to_run: Type,
                                 num_partitions: int,
                                 df: DataFrame,
                                 *args,
                                 **kwargs) -> DataFrame:
    """
    Runs the Threaded Processing Process
    Args:
        spark (SparkSession): The SparkSession
        class_to_run (Type): The class to run
        num_partitions (int): total number of partitions to iterate over 
        df (DataFrame): The partitioned df 
        *args: The args associated with class.
        **kwargs: The kwargs associated with class, if any.
    Returns:
       DataFrame: The unionised DataFrame
    """
    partitioned_dfs = []

    with ThreadPoolExecutor(max_workers=num_partitions) as executor:
        futures = [
            executor.submit(process_partition, spark, i, class_to_run, df, *args, **kwargs)
            for i in range(num_partitions)
        ]

        for future in as_completed(futures):
            result_df = future.result()
            partitioned_dfs.append(result_df)

    if partitioned_dfs:
        final_df = reduce(DataFrame.unionByName, partitioned_dfs)
        return final_df
    return spark.createDataFrame([], args[0].schema)


def runner(spark: SparkSession,
           class_to_run: Type,
           *args,
           **kwargs) -> DataFrame:
    """
    Args:
        spark (SparkSession): The SparkSession
        class_to_run (Type): The class to run
        *args: The args associated with class.
        **kwargs: The kwargs associated with class, if any.
    Returns:
        DataFrame: The returned DataFrame from the class to run.
    """
    df = args[0]  # Assuming the first positional argument is always the DataFrame
    total_rows = df.count()
    rows_per_partition = 100
    num_partitions = total_rows // rows_per_partition + (1 if total_rows % rows_per_partition else 0)
    df = df.repartition(num_partitions)
    return run_multithreaded_processing(spark, class_to_run, num_partitions, df, *args, **kwargs)


In [ ]:
"""
STEP 2
Create Enum class df and join it on to main driver df. 
Then create conditions 
"""

from src.data_generator.conditions_creator import ConditionsCreator
# from src.utils.thread_operations import runner
from src.constants.type_constants import DepartmentTypes

enum_values = [e.name for e in DepartmentTypes]

enum_df = spark.createDataFrame(enum_values, StringType()).toDF("admission_type")

selected_conditions_df = runner(spark, ConditionsCreator, base_table_df, joined_tbl, enum_df)


In [ ]:
selected_conditions_df.show()

In [ ]:
new_conditions_df = base_table_df.join(selected_conditions_df, on="unique_id", how="left")

In [ ]:
new_conditions_df.show().where("unique_id")

In [ ]:
base_table_df.filter(col("unique_id") == "T_Godfrey|2024-01-27").show()

In [ ]:
from pyspark.sql.functions import col, count

counts_df = new_conditions_df.groupBy("unique_id").agg(count("unique_id").alias("count")).filter(col("count") > 1)


In [ ]:
counts_df.show()

In [ ]:
from utils.util_funcs import get_row_count

get_row_count(selected_conditions_df)

In [ ]:
selected_conditions_df.show()

In [ ]:
# TODO:  
# tests to be chosen
# admission date to be chosen 
# hospital name 
# patient postcode 
# ethnicity distribution  
# _________________
# Long term 
# investigation which disease affects specific ethnicities
# blood type udf would need to consider ethnicity also eventually



In [ ]:
spark.stop()